In [94]:
import numpy as np
import matplotlib.pyplot as plt
import random
try:
    import importlib
    importlib.reload(h)
    importlib.reload(f)
    importlib.reload(d)
except NameError: # It hasn't been imported yet
    import helpers as h
    import functions as f
    import data_processing as d

# Data preprocessing and feature selections

## Data preprocessing

In [4]:
#For this to work, the data folder needs to be one level above the project folder and the folder name needs
#to be 'data'
data_folder = '../data/'
x_train, x_test, y_train, train_ids, test_ids = h.load_csv_data(data_folder)
features_name = np.genfromtxt('x_train.csv', delimiter=',', dtype=str, max_rows=1)

In [12]:
#load all the features names and remove the ID column
features_name = np.genfromtxt('../data/x_train.csv', delimiter=',', dtype=str, max_rows=1)[1:] 

## Feature selection

one paper on internet suggests to use these features : 

 _RFHYPE5, TOLDHI2, _CHOLCHK, _BMI5, SMOKE100, CVDSTRK3, DIABETE3, _TOTINDA, _FRTLT1, _VEGLT1, _RFDRHV5, HLTHPLN1, MEDCOST, GENHLTH, MENTHLTH, PHYSHLTH, DIFFWALK, SEX, _AGEG5YR, EDUCA, and INCOME2

 then, iterating through them, it removes the missing values, made the data binary when possible, removed the 'don't know, not sure', and ordinal (categorical) variables ares changed to 0,1,2,..., and renamed them

# Main

In [14]:
#Select the important features
features_list = ['_RFHYPE5', 'TOLDHI2', '_CHOLCHK', '_BMI5', 'SMOKE100', 'CVDSTRK3', 'DIABETE3', '_TOTINDA', '_FRTLT1', '_VEGLT1', '_RFDRHV5', 
                 'HLTHPLN1', 'MEDCOST', 'GENHLTH', 'MENTHLTH', 'PHYSHLTH', 'DIFFWALK', 'SEX', '_AGEG5YR', 'EDUCA', 'INCOME2']

#Create a mask to filter the data
mask = np.isin(features_name, features_list)

x_featured = x_train[:, mask]
x_test_featured = x_test[:, mask]

In [18]:
#remove all missing values on X and remove corresponding lines in Y and ids

x_train_featured_clean = x_featured[~np.isnan(x_featured).any(axis=1)]
x_test_featured_clean = x_test_featured[~np.isnan(x_test_featured).any(axis=1)]

y_train_clean = y_train[~np.isnan(x_featured).any(axis=1)]

train_ids_filtered = train_ids[~np.isnan(x_featured).any(axis=1)]
test_ids_filtered = test_ids[~np.isnan(x_test_featured).any(axis=1)]

print(x_train_featured_clean.shape, y_train_clean.shape, x_test_featured_clean.shape, train_ids_filtered.shape)


(257733, 21) (257733,) (85873, 21) (257733,)


### We want to clean the data for each feature, making them binary for yes/no, etc... and rename them

In [84]:
#train data

x_train_processed, y_train_processed, ids_train_processed = d.feature_processing(x_train_featured_clean, y_train_clean, train_ids_filtered)

In [85]:
#Test data
x_test_processed, y_test_processed, ids_test_processed = d.feature_processing(x_test_featured_clean,np.ones(len(x_test_featured_clean)), test_ids_filtered)

### Now that the preprocessing has been done, we can format the data to be used by the algorithms

In [86]:
tX_train = np.c_[np.ones((len(y_train_processed), 1)), x_train_processed]
tX_test = np.c_[np.ones((len(x_test_processed), 1)), x_test_processed]

## And then, we can run the algorithms

1. MSE gradient descent

In [87]:
#Compute gradient descent with MSE as loss function (see functions.py for the function)

initial_w = [random.choice([1, -1]) for i in range(len(tX_train[0]))]

loss_mse_gd, w_mse_gd = f.mean_squared_error_gd(y_train_processed, tX_train, initial_w, 200, 0.01)

Gradient Descent(199/199): loss=0.7212914239843792


In [88]:
#Test the model on the test sample. Do we need to standardize ?

y_test = tX_test.dot(w_mse_gd)
y_test_rounded = np.where(y_test > 0, 1, -1) #not sure about this line

print('weights = \n\n', w_mse_gd,'\n\n Loss = ', loss_mse_gd,'\n\n*****************************************************************************',
      ' \n\n Train sample : \n', 'Heart attack rate = ', np.count_nonzero(y_train_processed == 1)/len(y_train_processed), '\n \n Test sample : \n', 'Heart attack rate = ', np.count_nonzero(y_test_rounded == 1)/len(y_test_rounded))

weights = 

 [ 0.90047089  0.04269032  0.00819384 -0.00570958  0.81741539 -0.88493045
 -0.75961748  0.81475916  0.39126138 -0.72073252 -0.36517787 -0.01890852
  0.78925943  0.62340172 -0.74201228 -1.01042118 -0.03269382  0.94260673
  0.79201282 -0.85377879  0.67139906  0.6462009 ] 

 Loss =  0.7212914239843792 

*****************************************************************************  

 Train sample : 
 Heart attack rate =  0.09400982168649387 
 
 Test sample : 
 Heart attack rate =  0.2208738247610018


2. MSE SGD

In [89]:
loss_mse_sgd, w_mse_sgd = f.mean_squared_error_sgd(y_train_processed, tX_train, initial_w, 300, 0.001)

SGD iter. 0/299: loss=74.84638900499999, w0=0.9877651, w1=0.9388255
SGD iter. 1/299: loss=775.1313077505538, w0=1.0271384744489993, w1=1.0175722488979986
SGD iter. 2/299: loss=35.79789714120198, w0=1.035599904250304, w1=1.0260336786993032
SGD iter. 3/299: loss=106.42342469482571, w0=1.0210106325279165, w1=0.9530873200873664
SGD iter. 4/299: loss=61.69418266393934, w0=1.0321186640964652, w1=0.9641953516559153
SGD iter. 5/299: loss=47.20560646234296, w0=1.0418352073565287, w1=0.9933449814361058
SGD iter. 6/299: loss=91.51556078290206, w0=1.0553641068522677, w1=1.0068738809318447
SGD iter. 7/299: loss=5.95297457425344, w0=1.058814606692654, w1=1.010324380772231
SGD iter. 8/299: loss=2.0635423008864286, w0=1.0608461294224734, w1=1.01438742623187
SGD iter. 9/299: loss=6.202669127512969, w0=1.0643682506945231, w1=1.0284759113200692
SGD iter. 10/299: loss=5.179596578250881, w0=1.0611496806461163, w1=1.0188202011748486
SGD iter. 11/299: loss=20.224415781488126, w0=1.0675096202341137, w1=1.0251

In [90]:
y_test_sgd = tX_test.dot(w_mse_sgd)
y_test_rounded_sgd = np.where(y_test_sgd > 0, 1, -1)

print('weights = \n', w_mse_sgd,'\n Loss = ', loss_mse_sgd,'\n*****************************************************************************',
      ' \n Train sample : \n', 'Heart attack rate = ', np.count_nonzero(y_train_processed == 1)/len(y_train_processed), '\n \n Test sample : \n', 'Heart attack rate = ', np.count_nonzero(y_test_rounded_sgd == 1)/len(y_test_rounded_sgd))

weights = 
 [ 1.03735418  0.7527997  -0.0797142  -0.09515018  0.98474246 -0.90935238
 -0.96085035  1.04798106  0.98037329 -0.87226428 -0.53493934 -0.16245418
  1.03911024  1.00240917 -0.86897865 -0.95078661 -0.23604752  1.0048337
  0.99540721 -0.9601611   0.95268045  0.92983081] 
 Loss =  0.0011264968943467913 
*****************************************************************************  
 Train sample : 
 Heart attack rate =  0.09400982168649387 
 
 Test sample : 
 Heart attack rate =  0.2632219325274552


3. Least squares

In [91]:
loss_ls, w_ls = f.least_squares(y_train_processed, tX_train)

In [92]:
y_test_ls = tX_test.dot(w_ls)
y_test_ls = np.where(y_test_ls > 0, 1, -1)

print('weights = \n', w_ls,'\n Loss = ', loss_ls,'\n*****************************************************************************',
      ' \n Train sample : \n', 'Heart attack rate = ', np.count_nonzero(y_train_processed == 1)/len(y_train_processed), '\n \n Test sample : \n', 'Heart attack rate = ', np.count_nonzero(y_test_ls == 1)/len(y_test_ls))

weights = 
 [-4.23458311e-01  6.69229048e-02  1.84255656e-03 -2.88677208e-04
 -1.72692968e-02 -1.45978253e-02 -7.62584533e-02 -3.73059100e-01
 -4.33297730e-02 -1.06050366e-01  1.03039949e-03 -6.91952059e-03
 -9.97705263e-02 -4.55439527e-02 -6.73076101e-02  2.69260672e-02
  2.23087967e-02 -2.36533255e-01  3.86932707e-02 -6.06623971e-03
 -8.09010894e-03 -9.17165058e-03] 
 Loss =  0.14512298742909363 
*****************************************************************************  
 Train sample : 
 Heart attack rate =  0.09400982168649387 
 
 Test sample : 
 Heart attack rate =  0.0036185509994469465


4. Ridge regression

In [93]:
loss_ridge, w_ridge = f.ridge_regression(y_train_processed, tX_train, 0.1)

In [79]:
y_test_ridge = tX_test.dot(w_ridge)
y_test_ridge = np.where(y_test_ridge > 0.5, 1, 0)

print('weights = \n', w_ridge,'\n Loss = ', loss_ridge,'\n*****************************************************************************',
      ' \n Train sample : \n', 'Heart attack rate = ', np.count_nonzero(y_train_processed == 1)/len(y_train_processed), '\n \n Test sample : \n', 'Heart attack rate = ', np.count_nonzero(y_test_ridge == 1)/len(y_test_ridge))

weights = 
 [-0.05446848  0.01909874  0.00402915 -0.00136054 -0.01866968 -0.03347632
 -0.07015045 -0.10700854 -0.06969889 -0.07622882 -0.04981145 -0.02171525
 -0.07341898 -0.04008595 -0.06586933 -0.03631186  0.01438435 -0.01918326
 -0.03391057 -0.01702109 -0.01232494 -0.01216396] 
 Loss =  0.149955122105239 
*****************************************************************************  
 Train sample : 
 Heart attack rate =  0.09400982168649387 
 
 Test sample : 
 Heart attack rate =  0.0


5. ridge & least square polynomial regression

In [102]:
loss_log, w_log = f.logistic_regression(y_train_processed, tX_train, 100, 0.000001, initial_w)

y_test_log = tX_test.dot(w_log)
print(y_test_log)
y_test_log = np.where(y_test_log > 0, 1, -1)

print('weights = \n', w_log,'\n Loss = ', loss_log,'\n*****************************************************************************',
        ' \n Train sample : \n', 'Heart attack rate = ', np.count_nonzero(y_train_processed == 1)/len(y_train_processed), '\n \n Test sample : \n', 'Heart attack rate = ', np.count_nonzero(y_test_log == 1)/len(y_test_log))

Gradient Descent(0/99): loss=-1903764.6888709585, w0=0.8285066393687142, w1=0.5788726652137013
Gradient Descent(1/99): loss=-6393559.696314749, w0=0.6739094924049219, w1=0.22137822314805816
Gradient Descent(2/99): loss=-10119382.338791667, w0=0.5193124923635676, w1=-0.13611577696355825
Gradient Descent(3/99): loss=-13845203.169172822, w0=0.3647154923635435, w1=-0.49360977696361913
Gradient Descent(4/99): loss=-17571023.999199484, w0=0.21011849236354352, w1=-0.8511037769636192
Gradient Descent(5/99): loss=-21296844.829225928, w0=0.055521492363543534, w1=-1.2085977769636191
Gradient Descent(6/99): loss=-25022665.65925239, w0=-0.09907550763645645, w1=-1.5660917769636191
Gradient Descent(7/99): loss=-28748486.48927884, w0=-0.25367250763645643, w1=-1.923585776963619
Gradient Descent(8/99): loss=-32474307.3193053, w0=-0.4082695076364564, w1=-2.281079776963619
Gradient Descent(9/99): loss=-36200128.149331756, w0=-0.5628665076364564, w1=-2.638573776963619
Gradient Descent(10/99): loss=-3992594